In [2]:
import os
import re

try:
	os.chdir(os.path.join('/home/andrey/proj/OpenKE/'))
	print(os.getcwd())
except:
	pass

from tools.dataset_tools import Dataset

import pandas as pd
import numpy as np
from tools.tools import get_dirs, write_to_pkl, load_file, restore_model
from tools.dataset_tools import Dataset
from tqdm import tqdm
from collections import Counter

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity, euclidean_distances, pairwise_distances

import gensim

# import config

pd.set_option('display.max_colwidth', 400)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)

get_ipython().magic(u'load_ext autoreload')
get_ipython().magic(u'autoreload 2')

/home/andrey/proj/OpenKE


In [2]:
timestamp_sfe = '1904121223'
dataset = 'NELL186'
model = 'Analogy'
timestamp_emb = '1904121223'

project_folder = os.path.expanduser('~') + '/proj/XKE_results/{}/{}/{}/'.format(dataset, model, timestamp_emb)
load_from_splits = project_folder + 'splits/g_2negrate_bern__pra/'
feature_folder = project_folder + 'sfe_features/{}/'.format(timestamp_sfe)

d = Dataset(dataset)
d.load_true_sets()
# d.build_complete_true_set()
d.build_graph()

Loaded NELL186 Dataset with 14463 entities and 372 relations.

Loading NELL186 true facts...  Done!

Train set has 31134 triples
Test set has 5000 triples
Valid set has 5000 triples

Building NELL186 graph... Done!

Graph built with 72268 edges.


In [3]:
def show_most_similar(id, sim_model, top_n = 20):

    print('Query: {}'.format(d.names_dict[id]))

    similars = sim_model.most_similar(id, topn = 3 * top_n)

    return pd.DataFrame([(a[1], a[0], d.names_dict[a[0]]) for a in similars])# if a[0][0] == id[0]])


In [11]:
tt = d.names_dict['r0']
re.split("\s+", re.sub('[[\W+, _]', ' ', tt, flags=re.UNICODE))[1:]

['academic', 'field', 'such', 'as', 'academic', 'field']

## Pre-trained Word2Vec

In [3]:
kv = gensim.models.KeyedVectors.load_word2vec_format('word2vec/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [4]:
kv_model = gensim.models.keyedvectors.WordEmbeddingsKeyedVectors(300)

In [10]:
kv_model.add('plays-for', (kv.get_vector('plays') + kv.get_vector('for')) / 2)
kv_model.add('plays', kv.get_vector('plays'))
kv_model.add('coworker', kv.get_vector('coworker'))
kv_model.add('gender', kv.get_vector('gender'))

In [23]:
print(kv_model.similarity('plays', 'plays-for'))
print(kv_model.similarity('plays', 'plays'))

0.8982142131237828
0.9999999999999999


In [24]:
print(kv_model.similarity('plays', 'coworker')) 
print(kv_model.similarity('plays', 'gender'))

0.020877692663807103
0.10202353790960841


0.074974924325943

In [19]:
(kv.similarity('plays', 'for') + kv.similarity('plays', 'plays') + kv.similarity('plays', 'plays')) / 3

0.704027016957601

In [ ]:
test = d.names_dict['r3']
test = test + '&$#"@!.,<>|()] \'23 +Andrey'
print(test)

re.sub('[\W+, _]', ' ', test)

In [ ]:
#For FB15K237
kv_model = gensim.models.keyedvectors.WordEmbeddingsKeyedVectors(300)
not_found = []
for idx in tqdm(d.names_dict.keys()):
    description = d.names_dict[idx]
    if idx[0] == 'i':
        description = description[2:]
    if type(description) == str:
        words = (re.split("\s+", re.sub('[[\W+, _]', ' ', description, flags=re.UNICODE)))
    vectors = np.zeros(300)
    counter = 0
    for word in words:
        try:
            vectors += kv.get_vector(word)
            counter +=1
        except:
            continue
    if counter == 0: not_found.append(idx)
    kv_model.add(idx, vectors / counter)

In [12]:
#For NEEL186
kv_model = gensim.models.keyedvectors.WordEmbeddingsKeyedVectors(300)
not_found = []
for idx in tqdm(d.names_dict.keys()):
    description = d.names_dict[idx]
    if idx[0] == 'i':
        description = description[2:]
    if type(description) == str:
        words = (re.split("\s+", re.sub('[[\W+, _]', ' ', description, flags=re.UNICODE)))[1:]
    vectors = np.zeros(300)
    counter = 0
    for word in words:
        try:
            vectors += kv.get_vector(word)
            counter +=1
        except:
            continue
    if counter == 0: not_found.append(idx)
    kv_model.add(idx, vectors / counter)

100%|██████████| 14835/14835 [00:10<00:00, 1391.31it/s]


In [13]:
len(not_found)

0

In [ ]:
clean_not_found = [x[1:] for x in not_found]
clean_not_found

In [ ]:
mids = pd.read_csv('~/proj/OpenKE/benchmarks/FB15K237/mids2id.txt', sep='\t')

In [ ]:
mids[mids['id'].isin(clean_not_found)]

In [ ]:
show_most_similar('r0', kv_model, top_n = 40)

In [17]:
kv_model.save('word2vec/NELL186_Google_news_d300.model')

In [ ]:
kv = gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.load('word2vec/fb15k237_Google_news_d300.model')

In [16]:
show_most_similar('r0', kv_model, top_n = 40)

Query: concept:academic_field_such_as_academic_field


,0,1,2
0,1.000000,ir0,i*concept:academic_field_such_as_academic_field
1,0.875139,e6,concept:academic_field:business
2,0.863758,e12013,concept:academic_field:medical
3,0.856938,e57,concept:academic_field:program
4,0.855698,e78,concept:academic_field:technology
5,0.853145,e67,concept:academic_field:services
6,0.847640,e40,concept:academic_field:management
7,0.847046,e34,concept:academic_field:history
8,0.846941,e11961,concept:academic_field:health
9,0.844219,e64,concept:academic_field:science
